In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import *

In [2]:
def load_rpt(path):
    x, y = (13, 2)
    df = pd.read_excel(path, sheet_name='AI060')
    index = df.iloc[x:, 0].str.replace('\r\n', '').str.replace('_x000D_\n', '').values
    columns = df.iloc[9, y:].values
    rpt = pd.DataFrame(df.iloc[x:, y:].values, index=index, columns=columns).fillna(0).astype(float)
    return rpt

In [3]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('data/보유리스크율_자동차/20210805')
BASE_YYMM = '201912'
BASE_YYYY = BASE_YYMM[:4]

In [4]:
# 데이터 불러오기
자동차_원수_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'자동차_원수_직전3년연간경과보험료_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_ELP_PRM': float})
자동차_비례출재_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전3년연간경과보험료_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'P_RN_ELP_PRM': float})
자동차_원수_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'자동차_원수_직전3년연간손해액_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_LOSS': float})
자동차_비례출재_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'자동차_비례출재_직전3년연간손해액_{BASE_YYMM}.xlsx',
    dtype={'FY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'P_RN_LOSS': float})
자동차_특약보종별_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'자동차_특약보종별_직전1년경과보험료_{BASE_YYMM}.xlsx',
    dtype={'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_ELP_PRM_1YR': float, 'RN_ELP_PRM_1YR': float})
자동차_특약보종별_직전1년손해액 = pd.read_excel(FILE_PATH / f'자동차_특약보종별_직전1년손해액_{BASE_YYMM}.xlsx',
    dtype={'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_LOSS_1YR': float, 'RN_LOSS_1YR': float})
자동차_상품군정보 = pd.read_excel(FILE_PATH / '자동차_상품군정보.xlsx', dtype={'PDGR_CD': str, 'BSC_CVR_CD': str})
자동차_특약보종별_경과보험료 = pd.read_excel(FILE_PATH / f'자동차_특약보종별_경과보험료_{BASE_YYMM}.xlsx',
    dtype={'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_ELP_PRM': float, 'RN_ELP_PRM': float})
자동차_특약보종별_손해액 = pd.read_excel(FILE_PATH / f'자동차_특약보종별_손해액_{BASE_YYMM}.xlsx',
    dtype={'UY': str, 'BSC_CVR_CD': str, 'PDGR_CD': str, 'OGL_LOSS': float, 'RN_LOSS': float})
산업_손해율_변동계수 = pd.read_excel(FILE_PATH / f'산업_손해율_변동계수_{BASE_YYMM}.xlsx')
자동차_직전1년마감실적 = load_rpt(FILE_PATH / f'업무보고서_{BASE_YYMM}.xlsx')
자동차_1년전1년마감실적 = load_rpt(FILE_PATH / f'업무보고서_{int(BASE_YYMM)-100}.xlsx')
자동차_2년전1년마감실적 = load_rpt(FILE_PATH / f'업무보고서_{int(BASE_YYMM)-200}.xlsx')
자동차_위험계수 = pd.read_excel(FILE_PATH / f'자동차_위험계수_{BASE_YYMM}.xlsx')
자동차_KICS분류 = pd.read_excel(FILE_PATH / '자동차_KICS분류.xlsx')['BOZ_CD'].to_numpy()

In [5]:
# 위험계수
def get_comb_raio(data: pd.DataFrame) -> pd.DataFrame:
    # 'E': '경과보험료', 'IA': '발생손해액', 'QA': '순사업비'
    return data \
        .query('~BOZ_CD.isna()', engine='python') \
        .groupby('BOZ_CD')[['E', 'IA', 'QA']].sum() \
        .eval('합산비율 = (IA+QA)/E')[['합산비율']]

자동차_직전1년마감실적_가공 = get_comb_raio(자동차_직전1년마감실적.assign(BOZ_CD = lambda x: 자동차_KICS분류)) \
    .rename(columns={'합산비율': '합산비율_직전1년'}) \
    .reset_index()
자동차_1년전1년마감실적_가공 = get_comb_raio(자동차_1년전1년마감실적.assign(BOZ_CD = lambda x: 자동차_KICS분류)) \
    .rename(columns={'합산비율': '합산비율_1년전1년'}) \
    .reset_index()
자동차_2년전1년마감실적_가공 = get_comb_raio(자동차_2년전1년마감실적.assign(BOZ_CD = lambda x: 자동차_KICS분류)) \
    .rename(columns={'합산비율': '합산비율_2년전1년'}) \
    .reset_index()

자동차_위험계수_가공 = 자동차_2년전1년마감실적_가공 \
    .merge(자동차_1년전1년마감실적_가공, on='BOZ_CD', how='outer') \
    .merge(자동차_직전1년마감실적_가공, on='BOZ_CD', how='outer') \
    .merge(자동차_위험계수, on='BOZ_CD', how='outer') \
    .eval('COMB_RATIO = (합산비율_직전1년 + 합산비율_1년전1년 + 합산비율_2년전1년)/3') \
    .assign(RSK_COEF_PRM = lambda x: np.fmax(x.BSE_RSK_COEF_PRM*0.7, x.BSE_RSK_COEF_PRM+(x.COMB_RATIO-x.BSE_COMB_RATIO)*0.5)) \
    [['BOZ_CD', 'COMB_RATIO', 'RSK_COEF_PRM', 'RSK_COEF_RSV']]

In [6]:
pd.options.display.float_format = '{:,.3f}'.format

# 직전3년평균손해율
## TODO: 비비례출재실적 미반영 (반영 여부 추후 검토)
자동차_원수_직전3년연간경과보험료_가공 = 자동차_원수_직전3년연간경과보험료.copy()
자동차_원수_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전3년연간경과보험료_가공, 자동차_상품군정보)
자동차_원수_직전3년연간경과보험료_집계 = 자동차_원수_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_ELP_PRM']].sum()

자동차_비례출재_직전3년연간경과보험료_가공 = 자동차_비례출재_직전3년연간경과보험료.copy()
자동차_비례출재_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전3년연간경과보험료_가공, 자동차_상품군정보)
자동차_비례출재_직전3년연간경과보험료_집계 = 자동차_비례출재_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['P_RN_ELP_PRM']].sum()

자동차_원수_직전3년연간손해액_가공 = 자동차_원수_직전3년연간손해액.copy()
자동차_원수_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_원수_직전3년연간손해액_가공, 자동차_상품군정보)
자동차_원수_직전3년연간손해액_집계 = 자동차_원수_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_LOSS']].sum()

자동차_비례출재_직전3년연간손해액_가공 = 자동차_비례출재_직전3년연간손해액.copy()
자동차_비례출재_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_비례출재_직전3년연간손해액_가공, 자동차_상품군정보)
자동차_비례출재_직전3년연간손해액_집계 = 자동차_비례출재_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['P_RN_LOSS']].sum()

자동차_직전3년평균손해율 = 자동차_원수_직전3년연간경과보험료_집계 \
    .merge(자동차_비례출재_직전3년연간경과보험료_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(자동차_원수_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(자동차_비례출재_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer')
자동차_직전3년평균손해율[['OGL_ELP_PRM', 'P_RN_ELP_PRM', 'OGL_LOSS', 'P_RN_LOSS']] = 자동차_직전3년평균손해율[['OGL_ELP_PRM', 'P_RN_ELP_PRM', 'OGL_LOSS', 'P_RN_LOSS']].fillna(0)
자동차_직전3년평균손해율.eval('RET_ELP_PRM = OGL_ELP_PRM - P_RN_ELP_PRM', inplace=True)
자동차_직전3년평균손해율.eval('RET_LOSS = OGL_LOSS - P_RN_LOSS', inplace=True)
자동차_직전3년평균손해율.eval('LOSS_RATIO = RET_LOSS/RET_ELP_PRM', inplace=True)
자동차_직전3년평균손해율 = 자동차_직전3년평균손해율.groupby('BOZ_CD', as_index=False)['LOSS_RATIO'].mean()

# 특약보종별 직전1년실적
자동차_특약보종별_직전1년경과보험료_가공 = 자동차_특약보종별_직전1년경과보험료.copy()
자동차_특약보종별_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_특약보종별_직전1년경과보험료_가공, 자동차_상품군정보)
자동차_특약보종별_직전1년경과보험료_가공 = 자동차_특약보종별_직전1년경과보험료_가공.groupby(['BOZ_CD', 'UY'], as_index=False)[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR']].sum()

자동차_특약보종별_직전1년손해액_가공 = 자동차_특약보종별_직전1년손해액.copy()
자동차_특약보종별_직전1년손해액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_특약보종별_직전1년손해액_가공, 자동차_상품군정보)
자동차_특약보종별_직전1년손해액_가공 = 자동차_특약보종별_직전1년손해액_가공.groupby(['BOZ_CD', 'UY'], as_index=False)[['OGL_LOSS_1YR', 'RN_LOSS_1YR']].sum()

자동차_특약보종별_직전1년실적 = 자동차_특약보종별_직전1년경과보험료_가공 \
    .merge(자동차_특약보종별_직전1년손해액_가공, on=['BOZ_CD', 'UY'], how='outer')
자동차_특약보종별_직전1년실적[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR', 'OGL_LOSS_1YR', 'RN_LOSS_1YR']] = 자동차_특약보종별_직전1년실적[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR', 'OGL_LOSS_1YR', 'RN_LOSS_1YR']].fillna(0)
자동차_특약보종별_직전1년실적_가공 = []
for boz_cd in ['B001', 'B002', 'B003', 'B004', 'B005', 'B006', 'B007']:
    STRT_YYYY = str(int(BASE_YYYY) - 5)
    자동차_특약보종별_직전1년실적_가공.append(자동차_특약보종별_직전1년실적.query('BOZ_CD == @boz_cd').query('UY > @STRT_YYYY'))
자동차_특약보종별_직전1년실적_가공 = pd.concat(자동차_특약보종별_직전1년실적_가공, axis=0).reset_index(drop=True)

# 특약보종별 실적
자동차_특약보종별_경과보험료_가공 = 자동차_특약보종별_경과보험료.copy()
자동차_특약보종별_경과보험료_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_특약보종별_경과보험료_가공, 자동차_상품군정보)
자동차_특약보종별_경과보험료_가공 = 자동차_특약보종별_경과보험료_가공.groupby(['BOZ_CD', 'UY'], as_index=False)[['OGL_ELP_PRM', 'RN_ELP_PRM']].sum()

자동차_특약보종별_손해액_가공 = 자동차_특약보종별_손해액.copy()
자동차_특약보종별_손해액_가공['BOZ_CD'] = clsf_boz_cd_c(자동차_특약보종별_손해액_가공, 자동차_상품군정보)
자동차_특약보종별_손해액_가공 = 자동차_특약보종별_손해액_가공.groupby(['BOZ_CD', 'UY'], as_index=False)[['OGL_LOSS', 'RN_LOSS']].sum()

자동차_특약보종별_실적 = 자동차_특약보종별_경과보험료_가공 \
    .merge(자동차_특약보종별_손해액_가공, on=['BOZ_CD', 'UY'], how='outer')
자동차_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 자동차_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)
자동차_특약보종별_실적_가공 = []
for boz_cd in ['B001', 'B002', 'B003', 'B004', 'B005', 'B006', 'B007']:
    STRT_YYYY = str(int(BASE_YYYY) - 5)
    자동차_특약보종별_실적_가공.append(자동차_특약보종별_실적.query('BOZ_CD == @boz_cd').query('UY > @STRT_YYYY'))
자동차_특약보종별_실적_가공 = pd.concat(자동차_특약보종별_실적_가공, axis=0).reset_index(drop=True)

# 특약보종별 직전1년실적, 실적 집계
자동차_특약보종별_실적_집계 = 자동차_특약보종별_실적_가공 \
    .merge(자동차_특약보종별_직전1년실적_가공, on=['BOZ_CD', 'UY'], how='outer')

In [7]:
pd.options.display.float_format = '{:,.0f}'.format

# 보유리스크율
## 위험계수적용법
자동차_보유리스크율_위험계수적용법 = []
for boz_cd in 자동차_특약보종별_직전1년실적['BOZ_CD'].unique():
        prem = 자동차_특약보종별_직전1년실적_가공.query('BOZ_CD == @boz_cd').groupby('UY', as_index=False)[['OGL_ELP_PRM_1YR', 'RN_ELP_PRM_1YR']].sum()
        if len(prem) == 0: continue
        loss_ratio = 자동차_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
        rsk_coef = 자동차_위험계수_가공.query('BOZ_CD == @boz_cd')['RSK_COEF_PRM'].values[0]
        자동차_보유리스크율_위험계수적용법_특약보종별 = get_ret_risk_rate_by_risk_coef_c(boz_cd, prem, loss_ratio, rsk_coef)
        자동차_보유리스크율_위험계수적용법.append(자동차_보유리스크율_위험계수적용법_특약보종별)
자동차_보유리스크율_위험계수적용법 = pd.concat(자동차_보유리스크율_위험계수적용법, axis=0)
자동차_보유리스크율_위험계수적용법.query('OGL_ELP_PRM_1YR > 0', inplace=True)

In [9]:
pd.options.display.float_format = '{:,.0f}'.format

## 손해율분포법
자동차_보유리스크율_손해율분포법 = []
for boz_cd in 자동차_특약보종별_실적_집계['BOZ_CD'].unique():
    for uy in 자동차_특약보종별_실적_집계['UY'].unique():
        ogl_elp_prm_tty = 자동차_특약보종별_실적_집계.query('UY == @uy')['OGL_ELP_PRM'].sum()
        rn_elp_prm_tty = 자동차_특약보종별_실적_집계.query('UY == @uy')['RN_ELP_PRM'].sum()
        ogl_loss_tty = 자동차_특약보종별_실적_집계.query('UY == @uy')['OGL_LOSS'].sum()
        rn_loss_tty = 자동차_특약보종별_실적_집계.query('UY == @uy')['RN_LOSS'].sum()
        ogl_1yr_elp_prm_tty_boz = 자동차_특약보종별_실적_집계.query('UY == @uy').query('BOZ_CD == @boz_cd')['OGL_ELP_PRM_1YR'].sum()
        rn_1yr_elp_prm_tty_boz = 자동차_특약보종별_실적_집계.query('UY == @uy').query('BOZ_CD == @boz_cd')['RN_ELP_PRM_1YR'].sum()
        ogl_1yr_loss_tty_boz = 자동차_특약보종별_실적_집계.query('UY == @uy').query('BOZ_CD == @boz_cd')['OGL_LOSS_1YR'].sum()
        rn_1yr_loss_tty_boz = 자동차_특약보종별_실적_집계.query('UY == @uy').query('BOZ_CD == @boz_cd')['RN_LOSS_1YR'].sum()
        if ogl_1yr_elp_prm_tty_boz == 0: continue
        loss_ratio = 자동차_직전3년평균손해율.query('BOZ_CD == @boz_cd')['LOSS_RATIO'].values[0]
        cv = 산업_손해율_변동계수.query('BOZ_CD == @boz_cd')['CV'].values[0]
        
        ogl_risk, rn_risk = get_ret_risk_rate_by_loss_dist_c(
            ogl_elp_prm_tty, rn_elp_prm_tty, ogl_loss_tty, rn_loss_tty, 
            ogl_1yr_elp_prm_tty_boz, rn_1yr_elp_prm_tty_boz, ogl_1yr_loss_tty_boz, rn_1yr_loss_tty_boz, 
            loss_ratio, cv, uy
        )
        자동차_보유리스크율_손해율분포법.append([
            boz_cd, uy, '자동차보험비례특약', ogl_elp_prm_tty, rn_elp_prm_tty, ogl_loss_tty, rn_loss_tty,
            ogl_1yr_elp_prm_tty_boz, rn_1yr_elp_prm_tty_boz, ogl_1yr_loss_tty_boz, rn_1yr_loss_tty_boz,
            loss_ratio, loss_ratio*cv, '#', '#', '#', '#', '#', ogl_risk, rn_risk
            ])
자동차_보유리스크율_손해율분포법 = pd.DataFrame(자동차_보유리스크율_손해율분포법, columns=['BOZ_CD', 'UY', 'TTY_CD_GRP', 'OGL_ELP_PRM_TTY', 'RN_ELP_PRM_TTY', 'OGL_LOSS_TTY', 'RN_LOSS_TTY',
                'OGL_1YR_ELP_PRM_TTY_BOZ', 'RN_1YR_ELP_PRM_TTY_BOZ', 'OGL_1YR_LOSS_TTY_BOZ', 'RN_1YR_LOSS_TTY_BOZ', 
                'LOSS_RATIO_MEAN', 'LOSS_RATIO_STD', 'SLOPE', 'A', 'B', 'TOP', 'BOTTOM', 'OGL_RISK', 'RN_RISK'])
자동차_보유리스크율_손해율분포법.query('OGL_1YR_ELP_PRM_TTY_BOZ > 0', inplace=True)

In [11]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/자동차_보유리스크율_{now}.xlsx') as writer:
    자동차_보유리스크율_위험계수적용법.to_excel(writer, '자동차_보유리스크율_위험계수적용법', index=False)
    자동차_보유리스크율_손해율분포법.to_excel(writer, '자동차_보유리스크율_손해율분포법', index=False)